# Practical 7 - Part 2A
This second half of the lab explores the geometry of a single camera. In 2B the goal is to use a set of correspondance points to estimate a transformation matrix from a plane's 3D space to camera space and use that matrix to project some other points into camera space.

In this section, we'll work on building two components that we need for 2B, a method to estimate that transformation and a method that can project points into camera image space.

First we'll tackle the projection method, `projectiveCamera`. We want to find the image space coordinates, `XImCart`, of a set of 3D world coordinates, `XCart`, given a camera intrinsics matrix `K` and an extrinsics matrix `T`.

The second component is a method to estimate a Eucledian transformation, `TEst`, that takes us from a plane's 3D coordinate space to 3D camera space by utilizing a given set of points in camera image space, `XImCart`, and a set of corresponding points in world space, `XCart`. Essentially we want to compute the extrinsics matrix we can use in `projectiveCamera`.

Estimating the camera pose will involve calculating a homography, so you'll need to copy over your functions from part 1A/1B in the space provided.

## Import libraries 

In [3]:
%matplotlib inline
import os 
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [4]:
def projectiveCamera(K,T,XCart):
    ##TODO
    # The goal of this function is to project points in XCart through projective camera
    # defined by intrinsic matrix K and extrinsic matrix T. In essence, this function takes a set of points 
    # in 3D world space, XCart, and projects them into camera image space by applying the extrinsic matrix T 
    # and then applying the intrinsic matrix K.
    # 
    # There are three steps.
    # 1) Move from world space to camera space. 
    #            camera space points = extrinsics T * world space points 
    #
    # 2) Applying the intrinsics matrix to the camera space points after normalizing
    #           homogeneous image space points = K * normalized camera space points
    # 
    # 3) Move to image space cartesian points from image space homogeneous points, involves a 
    # normalization using the third row.
    
    
    # TODO: Convert Cartesian 3d points XCart to homogeneous coordinates XHom
    # Add a row of ones to convert 3D points into homogeneous coordinates
    XHom = np.concatenate((XCart, np.ones((1, XCart.shape[1]))), axis=0)
    
    # TODO: Apply extrinsic matrix to XHom, to move to the frame of reference of the camera
    # Transform points to camera space.
    # T is 4x4 matrix XHom is 4xN matrix
    XCamHom = np.matmul(T, XHom)

    # TODO: Project points into normalized camera coordinates xCamHom (remove 4th row)
    # Normalize by dividing by the fourth row (homogeneous to Cartesian conversion)
    XCamHom = XCamHom[:3, :] / XCamHom[3, :]

    # TODO: Move points to image coordinates xImHom by applying intrinsic matrix
    # Apply the intrinsic matrix K to camera coordinates
    xImHom = np.matmul(K, XCamHom)

    # TODO: Convert points back to Cartesian coordinates xImCart
    # Normalize by the third row to return to Cartesian coordinates
    XImCart = xImHom[:2, :] / xImHom[2, :]

    return XImCart


## Camera Projection

First we'll write up the function that can take us from 3D world space, `XCart`, to camera image space, `XImCart`, using an extrinsics matrix `T` and an intrinsics matrix `K` that are provided. The previous block houses this function.

The result here is the cartesian image space point coordinates, `XImCart`, of the 3D points `XCart`. If `XCart` represents a box in the world then we now know where the box's vertices would land in image space.

To verify that your solution is correct please compare your image space points to those in the comment.

Once they match, move on to the next bit - estimating a transformation! 

In [5]:
# We assume that the camera intrinsics matrix K is known and has values:
K = np.array([[640, 0, 320],
             [0, 640, 240],
             [0, 0, 1]])

# We will assume an object co-ordinate system with the Z-axis pointing upwards and the origin
# in the centre of the plane. There are five known points on the plane with coordinates (mm):
XCart = np.array([[-100, -100,  100,  100, 0],
                  [-100,  100,  100, -100, 0],
                  [   0,    0,    0,    0, 0]])

# We assume the correct transformation from the plane co-ordinate system to the
# camera co-ordinate system (extrinsic matrix) is:
T = np.array([[0.9851,  -0.0492,  0.1619,  46.00],
             [-0.1623,  -0.5520,  0.8181,  70.00],
             [0.0490,  -0.8324, -0.5518,  500.89],
             [0,        0,       0,       1]])
# T houses a rotation matrix and a translation matrix. The last row is for homogeneous point calculation.


# TODO: Use the general pin-hole projective camera model discussed in the lectures to estimate 
# where the four points on the plane will appear in the image.  Fill in the
# details of the function "projectiveCamera"
XImCart = projectiveCamera(K,T,XCart)

print(XImCart)
# Should be around:
# [267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
# [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]

[[267.4170882  230.95045427 531.42492013 482.36049098 378.77537982]
 [396.26814909 288.11435494 237.83410247 358.39940241 329.44079538]]


### You've implemented both of these functions in 1A and 1B already, so feel free to copy them in here. You'll need them for this next part.

In [22]:
def solveAXEqualsZero(A):
    # TODO: Write this routine - it should solve Ah = 0. You can do this using SVD. Consult your notes!
    # Hint: SVD will be involved.

    # Perform Singular Value Decomposition
    U, S, V = np.linalg.svd(A)
    
    # The solution to Ah = 0 is the last column of V (or last row of V transposed, V_t)
    h = (V.T)[:, -1]
    
    return h

In [23]:
def calcBestHomography(pts1Cart, pts2Cart):
    
    # This function should apply the direct linear transform (DLT) algorithm to calculate the best 
    # homography that maps the cartesian points in pts1Cart to their corresonding matching cartesian poitns 
    # in pts2Cart.
    
    # This function calls solveAXEqualsZero. Make sure you are wary of how to reshape h into a 3 by 3 matrix. 

    n_points = pts1Cart.shape[1]
    
    pts1Hom = np.concatenate((pts1Cart, np.ones((1,pts1Cart.shape[1]))), axis=0)
    
    # Construct matrix A (size: n_points * 2 , 9)
    A = np.zeros((n_points * 2, 9))

    u = pts1Hom[0,:]
    v = pts1Hom[1,:]
    x = pts2Cart[0,:]
    y = pts2Cart[1,:]

    for i in range (n_points):
        A[2*i] = np.array([0,0,0,-1*u[i], -1*v[i], -1, y[i]*u[i], y[i]*v[i], y[i]])
        A[2*i+1] = np.array([u[i], v[i], 1, 0, 0, 0, -1*x[i]*u[i], -1*x[i]*v[i], -1*x[i]])
    
    # Solve Ah = 0 using solveAXEqualsZero and get h.
    h= solveAXEqualsZero(A)
    
    # Reshape h into the matrix H, values of h go first into rows of H
    H = h.reshape((3,3))
    
    return H

In [45]:
# Read the next cell first for context!

def estimatePlanePose(XImCart,XCart,K):
    # The goal of this function is to estimate the pose of a plane relative to camera (extrinsic matrix)
    # given points in image space xImCart, points in 3D world space XCart, and an intrinsics matrix K.

    # TODO: Convert Cartesian image points XImCart to homogeneous representation XImHom
    XImHom = np.concatenate((XImCart, np.ones((1, XImCart.shape[1]))), axis=0)
    
    # TODO: Convert image co-ordinates XImHom to normalized camera coordinates XCamHom    
    XCamHom = np.matmul(np.linalg.inv(K), XImHom)

    
    # TODO: Estimate homography H mapping homogeneous (x,y) coordinates of positions
    # in real world to XCamHom (convert XCamHom to Cartesian, calculate the homography) -
    # use the routine you wrote for Practical 1B
    H = calcBestHomography(XCart[:2, :], XCamHom[:2, :])
          
    # TODO: Estimate first two columns of rotation matrix R from the first two
    # columns of H using the SVD. NOTE: You do not need to transpose v from linalg.svd    
    U,L,Vt = np.linalg.svd(H[:,0:2]) 
    rota = np.array([[1,0],[0,1],[0,0]])
    R = np.zeros((3,3))
    R12 = U@rota@Vt 

    # TODO: Estimate the third column of the rotation matrix by taking the cross
    # product of the first two columns
    R3 = np.cross(R12[:, 0], R12[:, 1])
    R = np.column_stack((R12, R3))
   
        
    # TODO: Check that the determinant of the rotation matrix is positive - if
    # not then multiply last column by -1.
    if np.linalg.det(R) <= 0:
        R[:, 2] *= -1

    
    # TODO: Estimate the translation t by finding the appropriate scaling factor k
    # and applying it to the third colulmn of H
    t = (H[:, 2] / (np.mean(H[:,:2]/R[:,:2]))).reshape(3,1)
    
    # TODO: Check whether t_z is negative - if it is then multiply t by -1 and
    # the first two columns of R by -1.
    if t[2] < 0:
        t *= -1
        R[:, :2] *= -1
            
    # TODO: Assemble transformation into matrix form
    T = np.hstack((R,t)) 
    T = np.vstack((T, np.array([0,0,0,1]))) 
    
    return T 

## Now the juicy bit: Estimating an Extrinsics Matrix, T

The problem: We are given an instrinsics matrix `K`, a set of 3D world points `XCart`, and a set of corresponding image space coordinates in `XImCart`. `K` and `XCart` have already been defined a few cells back and you've calculated `XImCart` by virtue of the exercise you've completed with camera projection. What we don't have is an extrinsics matrix, `T`. We need to estimate this and you'll need to fill in `estimatePlanePose` and return `TEst`.

Again you can start by negating the noise we add to XImCart to make sure you're on the right track.

In [46]:
# TODO: Add noise (standard deviation of one pixel in each direction) to the pixel positions
# to simulate having to find these points in a noisy image. Store the results back in xImCart
np.random.seed(0)  # For reproducibility
noise = np.random.normal(0, 1, XImCart.shape)  # Gaussian noise with std dev of 1 pixel
XImCartNoisy = XImCart + noise

# TODO: Now we will take the image points and the known positions on the card and estimate  
# the extrinsic matrix using the algorithm discussed in the lecture.  Fill in the details of 
# the function estimatePlanePose
TEst = estimatePlanePose(XImCartNoisy,XCart,K)

# If you have got this correct, TEst should closely resemble the groundtruth, T.
np.set_printoptions(precision=3)
print(TEst)
print("\n")
print(T)


[[ 9.860e-01 -5.890e-02  1.560e-01  4.759e+01]
 [-1.626e-01 -5.479e-01  8.206e-01  7.093e+01]
 [ 3.717e-02 -8.345e-01 -5.498e-01  5.074e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


[[ 9.851e-01 -4.920e-02  1.619e-01  4.600e+01]
 [-1.623e-01 -5.520e-01  8.181e-01  7.000e+01]
 [ 4.900e-02 -8.324e-01 -5.518e-01  5.009e+02]
 [ 0.000e+00  0.000e+00  0.000e+00  1.000e+00]]


We see that T represents the transformation from the plane coordinate system to the camera coordinate system. TEst is the estimated version of the extrinsic matrix. As shown in the results above, TEst and T are nearly identical, though not exactly the same due to the presence of noise.